In [1]:
import ipywidgets as widgets
import pandas as pd

In [3]:
def hello_world(b):
    
    url_data = r"https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/1d93ef50f3d8b878c59b45f64437077a726db8f5/2022_2023/data.csv"
    data_csv = pd.read_csv(url_data)
    print(data_csv)

button = widgets.Button(description='Update player list')
display(button)
button.on_click(hello_world)

Button(description='Update player list', style=ButtonStyle())

ParserError: Error tokenizing data. C error: Expected 1 fields in line 28, saw 509
